In [0]:
!rm -rf *
!rm -rf .git* .config
!ls -la

total 12
drwxr-xr-x 1 root root 4096 May 20 11:33 .
drwxr-xr-x 1 root root 4096 May 20 10:12 ..


In [0]:
!git clone https://github.com/manuhg/yolo_retrain.git .

In [0]:
!mkdir data_dir
!printf "car\ncart">data_dir/classes.txt

In [0]:
!python train.py -d data_dir